# YOLOv5 Custom Training 🚀

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Requirements

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 15300, done.
remote: Total 15300 (delta 0), reused 0 (delta 0), pack-reused 15300
Receiving objects: 100% (15300/15300), 14.19 MiB | 37.63 MiB/s, done.
Resolving deltas: 100% (10493/10493), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 6.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
cvxpy 1.2.3 requires setuptools<=64.0.2, but you have setuptools 67.6.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 3.6 MB/s eta 0:00:00
     

## Roboflow Dataset


In [ ]:
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

In [ ]:
!mkdir /content/datasets
!cd /content/datasets

In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="ZG73C7nrLF5yXM6t29E3")
project = rf.workspace("sappia").project("urbe_perception")
dataset = project.version(1).download("yolov5")

%cd /content/yolov5

In [ ]:
# version 2 --> preprocessing crop and mosaic augmentations
from roboflow import Roboflow
rf = Roboflow(api_key="ZG73C7nrLF5yXM6t29E3")
project = rf.workspace("sappia").project("urbe_perception")
dataset = project.version(2).download("yolov5")

%cd /content/yolov5

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to /content/datasets/urbe_perception-2 in yolov5pytorch:: 100%|██████████| 52012/52012 [00:08<00:00, 6410.36it/s]


/content/yolov5


## Train ⚡

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset location is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training

In [ ]:
!python train.py --img 640 --batch 32 --epochs 20 --data /content/datasets/urbe_perception-1/data.yaml --weights /content/drive/MyDrive/VISIOPE/Project/weights/yolov5m/best_v1.pt --cache --freeze 0 1 2 3 4 5 6 7 8 9

train: weights=/content/drive/MyDrive/VISIOPE/Project/weights/yolov5m/best_v1.pt, cfg=, data=/content/datasets/urbe_perception-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-119-g5543b89 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, c

## Evaluation 📏

### Test normally


> *FP16 half-precision can be used during inference by adding the **--half** flag!*

In [ ]:
!python val.py --weights /content/drive/MyDrive/VISIOPE/Project/weights/yolov5n/best_n.pt --data /content/datasets/urbe_perception-1/data.yaml --img 640 --half

### Post Pruning ✂

We repeat the above test with a pruned model by using the *torch_utils.prune()* command. We create an update version of *val.py* file to prune the model.

In [ ]:
!mv val.py val_old.py
!cp val_old.py val.py
!mv val.py val_prune.py
!mv val_old.py val.py

Add this lines of code to the new created *val_prune.py* file:
<a href="https://imgur.com/SoR1aJn"><img src="https://i.imgur.com/SoR1aJn.png" title="source: imgur.com" /></a>

In [ ]:
# Prune
from utils.torch_utils import prune
prune(model, 0.3)

In [ ]:
!python val_prune.py --weights /content/drive/MyDrive/VISIOPE/Project/weights/yolov5n/best_n.pt --data /content/datasets/urbe_perception-1/data.yaml --img 640 --half

In [ ]:
1/(2.7/1000)

370.3703703703703

## Video Inference 📹

In [ ]:
 !python detect.py --weights /content/drive/MyDrive/VISIOPE/Project/weights/yolov5n/best.pt --img 640 --conf 0.65 --source /content/drive/MyDrive/VISIOPE/Project/video/Streets_of_Rome.mp4

In [ ]:
!cp /content/yolov5/runs/detect/exp7/Streets_of_Rome.mp4 /content/drive/MyDrive/VISIOPE/Project/video/yolov5n